In [1]:
import requests
import json
import pandas as pd

In [2]:
# Instituts-Kürzel
acronym = "ISI"

# Jahr von (min) bis (max). Für die Einschränkung auf ein Jahr, bitte den gleichen Wert für date_min und date_max einfügen
#date = "2021"
date_min = "2020" 
date_max = "2022"


url = "https://publica-rest.fraunhofer.de/server/api/discover/search/objects?query=institute:("+acronym+")%20AND%20(dc.relation.ispartof:*)&configuration=researchoutputs&f.dateIssued=%5B"+date_min+"%20TO%20"+date_max+"%5D,equals"

r = requests.request("GET", url)
data = r.text
data = json.loads(data)
#print(data)
anzahl = data["_embedded"]["searchResult"]["page"]["totalElements"]
anzahl = int(anzahl)
print(anzahl)
pages = data["_embedded"]["searchResult"]["page"]["totalPages"]
pages = int(pages)
#print(pages)
object = data["_embedded"]["searchResult"]["_embedded"]["objects"]

pub_uuid = []
pub_name = []
pub_href = []
pub_journal = []
pub_journalAuth = []

object = data["_embedded"]["searchResult"]["_embedded"]["objects"]

for j in range(len(object)):
    pub_uuid.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"])
    pub_name.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["name"])
    pub_href.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_links"]["indexableObject"]["href"])
    pub_journal.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["value"])
    pub_journalAuth.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["authority"])

    
if pages > 1:
    for l in range(pages-1):
        url = data["_embedded"]["searchResult"]["_links"]["next"]["href"]
        r = requests.request("GET", url)
        #print(l)
        data = r.text
        data = json.loads(data)
        object = data["_embedded"]["searchResult"]["_embedded"]["objects"]
        for j in range(len(object)):
            pub_uuid.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["uuid"])
            pub_name.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["name"])
            pub_href.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_links"]["indexableObject"]["href"])
            pub_journal.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["value"])
            pub_journalAuth.append(data["_embedded"]["searchResult"]["_embedded"]["objects"][j]["_embedded"]["indexableObject"]["metadata"]["dc.relation.ispartof"][0]["authority"])
           


373


In [3]:
df = pd.DataFrame({'uuid':pub_uuid})
df['name'] = pub_name
df['href'] = pub_href
df['journal'] = pub_journal
df['journalAuth'] = pub_journalAuth

#print(df)

value_counts = df['journal'].value_counts(dropna=True, sort=True)
print(value_counts)

Technological forecasting and social change                  16
Energy policy                                                15
Environmental innovation and societal transitions            14
Energy research & social science                             10
Scientometrics                                                9
                                                             ..
DIW weekly report                                             1
Political psychology                                          1
South African journal of economic and management sciences     1
Energy and AI                                                 1
IEEE transactions on computational social systems             1
Name: journal, Length: 194, dtype: int64
